In [169]:
import json
import numpy as np

In [170]:
with open("results.json", "r") as f:
    data = json.load(f)

In [171]:
with open("/dfs/data/data/coco_wb/annotations/coco_wholebody_val.json") as f:
    template = json.load(f)

### Extract Information from Data

In [172]:
keypoints = []
bboxes = []
images = []
for annot, image in zip(data["annotations"], data["images"]):
    try:
        keypoints.append([arr['__ndarray__'] for arr in annot[0]["keypoints"][:23]])
        bboxes.append([arr for arr in annot[0]["bbox"]['__ndarray__']])
        images.append(image)
    except:
        continue

### Build json

In [173]:
json_images = []
for i,image in enumerate(images):
    json_images.append(
        dict({
            "file_name": image,
            "id": i
        }
        )
    )

In [174]:
json_annotations = []
for i, annot in enumerate(zip(bboxes, keypoints)):
    # convert bbox to coco format
    coco_bbox = [
        annot[0][0],
        annot[0][1],
        annot[0][2] - annot[0][0],
        annot[0][3] - annot[0][1],
    ]
    
    # convert keypoints to coco format
    keypoints = [[subarray[0], subarray[1], 2] if subarray[2] >= 0.3 else [0,0,0] for subarray in annot[1]]
    flattened_keypoints = [item for sublist in keypoints for item in sublist]
    
    num_keypoints = 0
    for i in range(2, len(flattened_keypoints), 3):
        if flattened_keypoints[i] == 2:
            num_keypoints += 1

    json_annotations.append(
        dict(
            {
                "num_keypoints": num_keypoints,
                "area": coco_bbox[2] * coco_bbox[3],
                "cvat_bbox": annot[0],
                "bbox": coco_bbox,
                "image_id": i,
                "id": i,
                "category_id": 1,
                "iscrowd": 0,
                "keypoints": flattened_keypoints,
            }
        )
    )

In [175]:
template["images"] = json_images
template["annotations"] = json_annotations

In [176]:
with open("coco_results.json", "w") as out:
    json.dump(template, out)